# NLP network project

In [10]:
import pandas as pd
import json
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re
import spacy
import igraph
import cairo

In [11]:
df =pd.read_csv("autos_seminuevos_mercado_libre1.csv")
df.head(1)

,nombre/0,precio/0,descripcion/0,caracteristicas/0/Marca,caracteristicas/0/Modelo,caracteristicas/0/Año,caracteristicas/0/Color,caracteristicas/0/Tipo de combustible,caracteristicas/0/Kilómetros,caracteristicas/0/Puertas,caracteristicas/0/Transmisión,caracteristicas/0/Motor,caracteristicas/0/Tipo de carrocería
0,"Chevrolet Tahoe 5 Pts Ltz, Ta, Piel, Qc, Dvd,...","700,000",En Nissan Imperio Coapa trabajamos para hacer ...,Chevrolet,Tahoe,2016,Blanco,Gasolina,87392 km,NaN,NaN,NaN,NaN


In [12]:
df.rename(columns={'nombre/0': 'nombre', 'precio/0': 'precio', 'caracteristicas/0/Marca': 'marca', 
                   'caracteristicas/0/Modelo': 'modelo', 'caracteristicas/0/Año': 'año', 'caracteristicas/0/Color': 'color',
                  'caracteristicas/0/Tipo de combustible' : 'combustible', 'caracteristicas/0/Kilómetros': 'kilometraje',
                  'caracteristicas/0/Puertas': 'puertas', 'caracteristicas/0/Transmisión': 'transmision', 'caracteristicas/0/Motor': 'motor',
                  'caracteristicas/0/Tipo de carrocería': 'tcarroceria', 'descripcion/0': 'descripcion'}, inplace = True)

In [13]:
#load spanish words from spacy
nlp=spacy.load('es_core_news_sm')
#identify spanish stopwords
stopwords = nlp.Defaults.stop_words


In [14]:
#lower case the whole "descripcion" column
df["descripcion"]=df["descripcion"].str.lower()
#remove special characters and stopwords
df["descripcion"]=df["descripcion"].replace({"[^\w\s]": "", "\d+": ""}, regex=True).apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df["descripcion_parsed"]=df["descripcion"].apply(nlp)

In [15]:
cv = CountVectorizer(ngram_range=(1,1))
X = cv.fit_transform(df["descripcion"])
Xc = (X.T * X) # matrix manipulation
Xc.setdiag(0) # set the diagonals to be zeroes as it's pointless to be 1
Xc

<7870x7870 sparse matrix of type '<class 'numpy.int64'>'
	with 1339890 stored elements in Compressed Sparse Column format>

In [16]:
names = cv.get_feature_names_out() # This are the entity names (i.e. keywords)

df1 = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)
df1
#df1.to_csv("co_occurrence.csv")

,aa,aac,aaceptamos,aaprobación,ab,abajo,abajosmartphone,abandono,abarth,abasolo,...,ópticos,óptimo,órdenes,únase,única,únicamente,únicamnete,único,únicos,útillibre
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aac,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaceptamos,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
aaprobación,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
únicamente,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
únicamnete,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
único,0,0,2,0,0,0,0,0,0,0,...,0,0,1,0,2,0,0,0,0,0
únicos,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
g = igraph.Graph.Adjacency(df1)
igraph.plot(g)

MemoryError: ("cairo returned CAIRO_STATUS_NO_MEMORY: b'out of memory'", 1)